# Cross Validation Index Generation

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut, StratifiedKFold
import pickle
import joblib
import os

## Load data

In [2]:
df_config = pd.read_csv('data/config.csv')
train_id = 0
df_config

,hash_id,label,raw_files_path,processed_file_folder,processed_file_path,cv_alg,cv_folds,cv_path,preproc_alg,pipeline_path,scaler_alg,train_data_path,train_trgt_path,model_path
0,6111500007297378247,Toy Data Classification with 10 StratifiedKFolds,data/raw,data,data/6111500007297378247_processed_data.csv,StratifiedKFolds,10,data/indexes,Não implementado para a aplicação!!!,data/pipelines,StandardScaler,data/6111500007297378247_train_data.csv,data/6111500007297378247_trgt_data.csv,data/models


### Load train data

In [3]:
df_data = pd.read_csv(df_config['train_data_path'][train_id])
df_trgt = pd.read_csv(df_config['train_trgt_path'][train_id])

In [4]:
df_data

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19
0,1.939775,-1.874778,-0.954075,-0.614433,-0.075541,2.156646,2.781148,0.664670,0.036624,-1.100478,-0.600619,-1.886817,1.749137,1.100532,-1.750317,3.586850,-0.851253,-1.758343,4.331898,-1.228460
1,0.212555,-0.840184,0.057779,-0.658165,0.747895,1.976136,-1.218683,2.098209,0.032722,-1.941232,-1.379431,-1.741510,-1.467274,-0.359222,-0.646826,1.221829,-3.193739,-1.231459,-4.084591,-0.121236
2,-0.600012,1.277222,-0.119119,-1.820902,1.340220,-4.064798,0.844581,-0.329720,-1.017783,3.595240,0.403389,1.837657,-0.999343,1.270964,-0.400716,-5.644099,1.500718,-1.751000,3.452837,-0.330061
3,1.249253,0.187247,0.005743,-1.262467,0.235543,-0.656909,0.886073,-3.209410,1.076622,1.271265,0.610868,3.111609,0.185904,2.239095,1.175438,0.087544,1.533219,0.500718,7.961663,-1.313997
4,0.425823,1.858765,0.626111,-4.170287,0.186934,-1.872444,1.188883,-2.172106,-0.067134,2.308339,0.091623,-0.958886,-0.932955,0.008028,-0.133388,-0.149368,3.828639,-0.405985,6.439644,1.921799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.648373,1.257027,-0.046447,-0.456077,-0.073366,-1.663773,-0.656953,0.609368,-1.326729,-3.670528,0.711640,-2.311737,0.474423,1.625338,-1.524986,0.697302,0.650684,5.693063,-4.362135,-1.855447
9996,-0.469769,0.349265,2.177472,2.821134,0.544254,-1.864935,-0.070637,-0.580827,2.115372,5.902980,-0.556782,-2.126881,-0.411713,-0.453700,-1.654114,-2.996429,3.389332,0.096562,-2.409423,-0.927629
9997,0.600509,-0.145892,0.240435,0.772653,1.619995,-3.297447,1.191270,-2.382692,0.816907,2.559541,0.611280,-1.283847,-0.718795,0.666438,-0.726815,0.015864,2.801846,0.444815,2.169056,3.748256
9998,1.252644,1.261386,0.806004,2.283086,1.396687,-0.781554,-3.993369,-1.913097,0.860037,2.893866,-0.746157,0.133769,-0.631262,-1.906922,-0.233603,-3.168554,-1.893054,1.176758,-6.304567,-1.021925


# Stratified K-Folds

In [5]:
from sklearn.model_selection import StratifiedKFold
import pickle
import os

file_path = df_config['cv_path'][train_id]
n_folds = df_config['cv_folds'][train_id]

cv = StratifiedKFold(n_splits=n_folds)

#data = df_train.drop(columns=['target']).values
#target = df_train['target'].values

data = df_data
target = df_trgt

for ifold, (trn_idx, val_idx) in enumerate(cv.split(data,target)):
    cv_name ='%s_%s_CV_fold_%i_of_%i_cv_indexes.pkl'%(df_config['hash_id'][train_id],
                                                       df_config['cv_alg'][train_id],
                                                       ifold, n_folds)
    with open(os.path.join(file_path,cv_name),'wb') as file_handler:
        pickle.dump([trn_idx,val_idx],file_handler)

In [6]:
data.shape

(10000, 20)

In [7]:
target.shape

(10000, 1)